In [ ]:
json_path = "./data/raw-train/tiger-coco.json"
img_dir = "./data/raw-train/"
img_dir_precise = "./data/raw-train/images"
crop_rois_dir = "./data/processed-train/cropped/"
processed_dir = "./data/processed-train/"

In [ ]:
from src.image_preprocessor import ImageProcessor, InplaceOption

image_processor = ImageProcessor("./data/raw-test/images")

In [ ]:
target_images = image_processor.select_random_images()
image_processor.normalize(target_images, out_dir=processed_dir, inplace=True, inplace_option=InplaceOption.HEMATOXYLIN)
image_processor.equalize_hist(inplace=True, out_dir="./data/processed-train/equalized_hist")

In [ ]:
image_processor.relabel_binary(masks_dir="./data/raw-test/masks", out_dir="./data/processed-test/masks_binary", target_class=2)

In [ ]:
image_processor.scale(masks_dirs=["./data/processed-test/masks_binary"], out_dir="./data/processed-test/scaled", inplace=True)

In [ ]:
image_processor.extract_patches(masks_dirs=["./data/processed-test/scaled/masks/"], out_dir="./data/processed-test/patches")

In [ ]:
from src.image_stats import ImageStats

img_stats = ImageStats("./data/processed-train/patches/images")
img_stats.get_image_stats(visual=True)

In [ ]:
std = img_stats.contrast_std()
michelson = img_stats.michelson_contrast()

In [1]:
from src.mask_generator import MaskGenerator, MaskOperations

mask_generator = MaskGenerator("./data/raw-train/images", json_file_path="./data/raw-train/tiger-coco.json")

In [2]:
mask_generator._fuse_masks(masks_dir="./data/processed-train/masks", out_dir="./data/processed-train/masks_fused", colorized_dir="./data/processed-train/masks_fused_color")

In [ ]:
from src.sample import Sample

sample = Sample(img_path="./data/processed-test/images_patch/09_4.png_(256_256).png", mask_path="./data/processed-test/masks_patch/09_4.png_(256_256).png")
sample.show(with_mask=True)